# 5.0 Modelling

---

# Contents

- [1.0 Arima Model](#1.0-ARIMA-Model)
- [2.0 Daily Data](#2.0-Daily-Data)
    - [1.1 Load Data](#2.1-Load-Data)
    - [1.2 Train Test Split](#2.2-Train-Test-Split)

In [1]:
# !pip install pmdarima

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import calendar

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA, ARMA, ARMAResults, ARIMAResults
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import r2_score, mean_squared_error
from pmdarima import auto_arima
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")


from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse


In [3]:
pd.set_option('display.max_columns', None)

---

# 1.0 ARIMA Model

The Arima model has 3 components:

+ Differencing Step - I - Integrated - Check for stationarity
+ Autoregressive Piece - AR - long term trends
+ Moving Average Piece - MA - Modelling sudden fluctuations

Each part has input into the model P,D,Q. I will use the previous Dickey Fuller 

+ D is the order of differencing we found using the Augmented Dickey-Fuller test.
+ P is the number of autoregressive terms in our model. PACF is used to estimate this.
+ Q is to do with looking at the moving average.
    + If PACF has a sharp cut off and lag-1 for the ACF is negative choose q to be the lag in the ACF before cut off.
    + If PACF does not have a sharp cut off or lag -1 ACF is not negative choose q = 0

Therefore based on the charts before I will use:
    
    + p = 1
    + d = 1
    + q = 0
    
However I will use auto_arima to help decide.

---

In [4]:
results = {'algo':'','name':'','date':'', 'time_frame':'','success':0,'RMSE':0, 'MSE':0, 'classification':'' }

##### 1.1 Load Data

In [5]:
daily = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/daily.csv', 
                    index_col='date', parse_dates=True)

In [6]:
daily.index

DatetimeIndex(['2000-11-13', '2000-11-14', '2000-11-15', '2000-11-16',
               '2000-11-17', '2000-11-20', '2000-11-21', '2000-11-22',
               '2000-11-23', '2000-11-24',
               ...
               '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-16',
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24'],
              dtype='datetime64[ns]', name='date', length=4921, freq=None)

In [7]:
daily.columns

Index(['open', 'high', 'low', 'close', 'gold_usd', 'gold_euro', 'marubozu',
       'marubozu+1', 'marubozu-1', 'marubozu-2', 'height', 'av_3_height',
       'wk_mv_av', 'mnth_mv_av', 'qtr_mv_av', 'vol', 'day-1_open',
       'day-2_open', 'day-3_open', 'day-1_high', 'day-2_high', 'day-3_high',
       'day-1_low', 'day-2_low', 'day-3_low', 'day-1_close', 'day-2_close',
       'day-3_close', 'day+1_open', 'day+1_high', 'day+1_low', 'day+1_close',
       'day+2_high', 'day+3_high', 'day+4_high', 'day+5_high', 'date+5',
       'target', 'double_height', 'select'],
      dtype='object')

In [8]:
#daily = daily.resample('B').agg({'open':'first','high':'max','low':'min', 'close':'last'})


In [9]:
daily.index

DatetimeIndex(['2000-11-13', '2000-11-14', '2000-11-15', '2000-11-16',
               '2000-11-17', '2000-11-20', '2000-11-21', '2000-11-22',
               '2000-11-23', '2000-11-24',
               ...
               '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-16',
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24'],
              dtype='datetime64[ns]', name='date', length=4921, freq=None)

In [10]:
daily['date+5'] = pd.to_datetime(daily['date+5'])

In [11]:
daily.loc[daily.index[1],'date+5']

Timestamp('2000-11-21 00:00:00')

In [12]:
type(daily['date+5'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [13]:
### Get correct hyper parameters

In [14]:
## Arima
auto_arima(daily['close'].dropna(), seasonal=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 4921
Model:               SARIMAX(0, 1, 0)   Log Likelihood               17044.750
Date:                Thu, 30 Jul 2020   AIC                         -34087.501
Time:                        18:50:49   BIC                         -34080.999
Sample:                             0   HQIC                        -34085.220
                               - 4921                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      5.732e-05   7.53e-07     76.145      0.000    5.58e-05    5.88e-05
===================================================================================
Ljung-Box (Q):                       30.36   Jarque-Bera (JB):              1507.98
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               0.69   Skew:                             0.00
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.71
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

---

## 1.1 Get patterns

In [15]:
daily_pattern = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/targets/daily_pattern.csv', 
                           parse_dates=True)

In [16]:
daily_pattern['pattern_end'] = pd.to_datetime(daily_pattern['pattern_end'])

In [17]:
daily_pattern.loc[1]

pattern_end   2001-04-04
Name: 1, dtype: datetime64[ns]

In [18]:
len(daily_pattern)

62

---

In [19]:
def create_train_test_split(date, time_series, model_info):
    test_end_date = time_series.loc[date,'date+5']
    
    train_test = time_series.loc[time_series.index <= test_end_date]
  
    target_value = time_series.loc[time_series.index == date,'double_height'].item()
    
    train_test.insert(0, 'target_price', target_value)
    
    model_info['signal'] = time_series.loc[date,'marubozu']
    
    train_test.insert(0, 'signal', model_info['signal'])
    
    model_info['start'] = len(train_test)-5
    model_info['end'] = len(train_test)-1
    
    model_info['train'] = train_test.iloc[:model_info['start']]
    model_info['test'] = train_test.iloc[model_info['start']:]

    return model_info

In [28]:
def train_arima(model_info, p=0, d=1, q=0):
#     exog=model_info['train']['wk_mv_av']
    
    exog = np.column_stack([model_info['train']['mnth_mv_av'], 
                            model_info['train']['wk_mv_av'],
                            model_info['train']['vol'],
                            model_info['train']['gold_euro'],
                            model_info['train']['gold_usd']])
    
    if model_info['signal'] == -1:
        model = ARIMA(model_info['train']['low'], exog=exog, order=(p,d,q))
    else:
        model = ARIMA(model_info['train']['high'], exog=exog, order=(p,d,q))

    results = model.fit()
    predictions = results.predict(start=model_info['start'], 
                                  end=model_info['end'], exog=exog,
                                  dynamic=True, 
                                  typ='levels').rename('ARIMA-0-1-0 Predictions')
    
    return results, predictions

In [29]:
def meet_threshold(row):
    if row['signal'] == -1 and row['low'] <= row['target_price']:
        return -1
    elif row['signal'] == 1 and row['high'] >= row['target_price']:
#         print(f"row high: {row['high']} >= row dbl height: {row['target_price']}" )
        return 1    
    else:
        return 0

In [30]:
def ml_decision(row):
    if row['direction'] == -1 and row['preds'] <= row['target_price']:
        return -1
    elif row['direction'] == 1 and row['preds'] >= row['target_price']:
        print(f"preds: {row['preds']} >= row target: {row['target_price']}" )
        return 1    
    else:
        return 0

In [31]:
def create_results_outcomes_dataframe(test, predictions):    
    outcomes = pd.DataFrame()
    outcomes['low'] = test['low']
    outcomes['high'] = test['high']
    outcomes['preds'] = predictions.values
    outcomes['target_price'] = test['target_price']
    outcomes['direction'] = test['signal']
    outcomes['correct_call'] = test.apply(meet_threshold, axis=1)

    return outcomes

In [32]:
def print_chart(outcomes):
    if model_info['signal'] == -1:
        outcomes['low'].plot(legend=False, figsize=(12,8))
    else:
        outcomes['high'].plot(legend=False, figsize=(12,8))

    outcomes['preds'].plot(legend=False);
    outcomes['target_price'].plot(legend=False);

In [33]:
def get_results(model_info):
        
    if model_info['signal'] == -1:
        mse = mean_squared_error(model_info['test']['low'], predictions)
        rmse_res = rmse(model_info['test']['low'], predictions)
    else:
        mse = mean_squared_error(model_info['test']['high'], predictions)
        rmse_res = rmse(model_info['test']['high'], predictions)       
    
    return rmse_res, mse

In [34]:
def classify(outcomes):
    
    if max(outcomes['direction']) == 1:
        
        if max(outcomes['correct_call']) == 0 and max(outcomes['ml_correct_call']) == 0:
            return 'tn'
        elif max(outcomes['correct_call']) == 1 and max(outcomes['ml_correct_call']) == 1:
            return 'tp'
        elif max(outcomes['correct_call']) == 0 and max(outcomes['ml_correct_call']) == 1:
            return 'fp'
        elif max(outcomes['correct_call']) == 1 and max(outcomes['ml_correct_call']) == 0:
            return 'fn'
        
    elif max(outcomes['direction']) == -1:
        
        if min(outcomes['correct_call']) == 0 and min(outcomes['ml_correct_call']) == 0:
            return 'tn'
        elif min(outcomes['correct_call']) == -1 and min(outcomes['ml_correct_call']) == -1:
            return 'tp'
        elif min(outcomes['correct_call']) == 0 and min(outcomes['ml_correct_call']) == -1:
            return 'fp'
        elif min(outcomes['correct_call']) == -1 and min(outcomes['ml_correct_call']) == 0:
            return 'fn'
        
    else:
        return 'ERROR'
    

In [55]:
count = 0

model_info = {"train":None,"test":None,"start":None,"end":None,"signal":None}
arima_results = []

for match in daily_pattern['pattern_end']:
    
    results_dict = {'name':None,'pattern':None,'date':None,
                   'time_frame':None,'RMSE':None,
                   'MSE':None, 'classification':None}
    
    results_dict['name'] = 'arima-0-1-0' + str(match)
    results_dict['strategy'] = 'Maribozu'
    results_dict['time_frame'] = 'daily'
    
#     if count <= 10:
#         count+=1
    model_info = create_train_test_split(match, daily, model_info)

    if len(model_info['train']) < 10:
        continue

#     print(f"len wk_av: {len(model_info['train']['wk_mv_av'])}, shape: {model_info['train']['wk_mv_av'].shape}")   
#     print(f"len low: {len(model_info['train']['low'])}, shape: {model_info['train']['low'].shape}")   

    results, predictions = train_arima(model_info)

    print(f"type: {type(predictions)}, len: {len(predictions)}")
    outcomes = create_results_outcomes_dataframe(model_info['test'], predictions)
    outcomes['ml_correct_call'] = outcomes.apply(ml_decision, axis=1)

    print(outcomes.head())
#     print_chart(outcomes)

    results_dict['RMSE'], results_dict['MSE'] = get_results(model_info)
    results_dict['classification'] = classify(outcomes)

    arima_results.append(results_dict)



type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                                                                          
2001-02-09  0.9135  0.9291  0.915590        0.9085         -1             0   
2001-02-12  0.9272  0.9332  0.915085        0.9085         -1             0   
2001-02-13  0.9174  0.9313  0.915508        0.9085         -1             0   
2001-02-14  0.9170  0.9222  0.913535        0.9085         -1             0   
2001-02-15  0.9029  0.9198  0.911635        0.9085         -1            -1   

            ml_correct_call  
date                         
2001-02-09                0  
2001-02-12                0  
2001-02-13                0  
2001-02-14                0  
2001-02-15                0  
type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                                                     

type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                                                                          
2006-04-13  1.2071  1.2135  1.208001        1.2052         -1             0   
2006-04-14  1.2102  1.2200  1.209490        1.2052         -1             0   
2006-04-17  1.2169  1.2290  1.211464        1.2052         -1             0   
2006-04-18  1.2227  1.2370  1.211873        1.2052         -1             0   
2006-04-19  1.2288  1.2396  1.212460        1.2052         -1             0   

            ml_correct_call  
date                         
2006-04-13                0  
2006-04-14                0  
2006-04-17                0  
2006-04-18                0  
2006-04-19                0  
type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                                                     

type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                                                                          
2009-07-01  1.4018  1.4201  1.401971        1.3958         -1             0   
2009-07-02  1.3927  1.4128  1.403701        1.3958         -1            -1   
2009-07-03  1.3952  1.4028  1.405489        1.3958         -1            -1   
2009-07-06  1.3876  1.3998  1.407091        1.3958         -1            -1   
2009-07-07  1.3881  1.4050  1.408711        1.3958         -1            -1   

            ml_correct_call  
date                         
2009-07-01                0  
2009-07-02                0  
2009-07-03                0  
2009-07-06                0  
2009-07-07                0  
type: <class 'pandas.core.series.Series'>, len: 5
preds: 1.4307549250460312 >= row target: 1.4304
preds: 1.43239300115302 >= row target: 1.4304
preds: 1.4340548757205738 >= row target: 1

2011-07-11                0  
type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                                                                          
2011-08-11  1.4103  1.4293  1.413822        1.4086         -1             0   
2011-08-12  1.4149  1.4317  1.415495        1.4086         -1             0   
2011-08-15  1.4263  1.4476  1.417201        1.4086         -1             0   
2011-08-16  1.4351  1.4471  1.418778        1.4086         -1             0   
2011-08-17  1.4324  1.4516  1.420376        1.4086         -1             0   

            ml_correct_call  
date                         
2011-08-11                0  
2011-08-12                0  
2011-08-15                0  
2011-08-16                0  
2011-08-17                0  
type: <class 'pandas.core.series.Series'>, len: 5
               low    high     preds  target_price  direction  correct_call  \
date                       

type: <class 'pandas.core.series.Series'>, len: 5
                low     high     preds  target_price  direction  correct_call  \
date                                                                            
2016-01-21  1.07777  1.09210  1.088318       1.08277         -1            -1   
2016-01-22  1.07882  1.08628  1.089995       1.08277         -1            -1   
2016-01-25  1.08059  1.08593  1.091684       1.08277         -1            -1   
2016-01-26  1.08186  1.08744  1.093313       1.08277         -1            -1   
2016-01-27  1.08527  1.09163  1.094953       1.08277         -1             0   

            ml_correct_call  
date                         
2016-01-21                0  
2016-01-22                0  
2016-01-25                0  
2016-01-26                0  
2016-01-27                0  
type: <class 'pandas.core.series.Series'>, len: 5
preds: 1.1387006956940307 >= row target: 1.1382499999999998
preds: 1.140395759512872 >= row target: 1.1382499999999998
pre

2019-02-07                0  
type: <class 'pandas.core.series.Series'>, len: 5
                low     high     preds  target_price  direction  correct_call  \
date                                                                            
2019-02-14  1.12498  1.13098  1.126442       1.12234         -1             0   
2019-02-15  1.12340  1.13248  1.128228       1.12234         -1             0   
2019-02-18  1.12930  1.13338  1.130016       1.12234         -1             0   
2019-02-19  1.12755  1.13574  1.131750       1.12234         -1             0   
2019-02-20  1.13249  1.13711  1.133504       1.12234         -1             0   

            ml_correct_call  
date                         
2019-02-14                0  
2019-02-15                0  
2019-02-18                0  
2019-02-19                0  
2019-02-20                0  


In [53]:
arima_results

[{'name': 'arima-0-1-02001-02-08 00:00:00',
  'pattern': None,
  'date': None,
  'time_frame': 'daily',
  'RMSE': 0.006971616737024272,
  'MSE': 4.860343992795697e-05,
  'classification': 'fn',
  'strategy': 'Maribozu'},
 {'name': 'arima-0-1-02001-04-04 00:00:00',
  'pattern': None,
  'date': None,
  'time_frame': 'daily',
  'RMSE': 0.006432884953439646,
  'MSE': 4.138200882419019e-05,
  'classification': 'tn',
  'strategy': 'Maribozu'},
 {'name': 'arima-0-1-02001-04-09 00:00:00',
  'pattern': None,
  'date': None,
  'time_frame': 'daily',
  'RMSE': 0.007587544746477939,
  'MSE': 5.757083527980498e-05,
  'classification': 'tp',
  'strategy': 'Maribozu'},
 {'name': 'arima-0-1-02001-08-20 00:00:00',
  'pattern': None,
  'date': None,
  'time_frame': 'daily',
  'RMSE': 0.0031454077646916375,
  'MSE': 9.893590006182445e-06,
  'classification': 'fp',
  'strategy': 'Maribozu'},
 {'name': 'arima-0-1-02002-02-20 00:00:00',
  'pattern': None,
  'date': None,
  'time_frame': 'daily',
  'RMSE': 0

In [37]:
def create_cm(arima_results):
    
    res_cm = [[0,0],
              [0,0]]
    
    for result in arima_results:
        res = result['classification']
        
        if res == 'tp':
            res_cm[0][0] += 1
        elif res == 'fp':
            res_cm[0][1] += 1
        elif res == 'fn':
            res_cm[1][0] += 1
        elif res == 'tn':
            res_cm[1][1] += 1
    
    return res_cm

In [38]:
cm = create_cm(arima_results)

In [40]:
cm_df = pd.DataFrame(cm, index=['pred_success', 'pred_non_success'], columns=['actual success', 'actual non_success'])
cm_df

,actual success,actual non_success
pred_success,14,6
pred_non_success,24,18


In [46]:
# Accuracy - how many did the model get right
# Total number of correct predictions / total number of predictions
(14+18)/(14+18+6+24)

0.5161290322580645

In [47]:
# Precision proportion of positive identifications that were actually correct
# True positives/ true positives + false positives)
14/(14+6)

0.7

In [50]:
# Recall - proportion od actual positives that were correctly defined
# True positives/ true positives + false negatives
14/(14+24)

0.3684210526315789

## 1.2 Train Test Split

In [ ]:
type(daily.loc[daily.index == daily_pattern.loc[10]['pattern_end']].index[0])

In [ ]:
daily.loc[daily.index == daily_pattern.loc[10]['pattern_end']].index[0]

In [ ]:
daily_pattern.index

In [ ]:
# Test 1 date out
curr_pattern = daily.loc[daily.index == daily_pattern.loc[10]['pattern_end']].index[0]
curr_pattern

In [ ]:
daily.index

In [ ]:
test_end_date = daily.loc[daily.loc[daily.index == daily_pattern.loc[10]['pattern_end']].index[0],'date+5']
test_end_date

In [ ]:
# daily.loc[daily.index == curr_pattern]

In [ ]:
train_test = daily.loc[daily.index <= test_end_date]
# train_test = daily.loc[daily.index <= '2004-2-28 00:00:00']

In [ ]:
# daily.loc[daily.index <= end_date]

In [ ]:
# daily.loc[daily.index == daily_pattern.loc[10]['pattern_end'],'double_height']

In [ ]:
target_value = daily.loc[daily.index == daily_pattern.loc[10,'pattern_end'],'double_height'].item()
target_value

In [ ]:
# def choose_exit_price(row, target_price, signal=-1):
#     if signal == -1:
#         return target_price
# #         return row['close'] - (row['height'] * 1)
#     else:
#         return target_price

# #         return row['close'] + (row['height'] * 1)

In [ ]:
train_test

In [ ]:
# train_test['double_height'] = train_test.apply(choose_exit_price, axis=1)
# train_test['double_height'] = daily.loc[daily.index == daily_pattern.loc[10,'pattern_end'],'double_height'].item()
#train_test.loc['double_height'] = [target_value for x in train_test.loc[:,['double_height']]]
train_test.insert(0, 'target_price', target_value)
# train_test.insert(0, 'signal', signal)

In [ ]:
signal = daily.loc[daily.index == daily_pattern.loc[10,'pattern_end'],'marubozu'].item()
signal

In [ ]:
train_test.head()

In [ ]:
[signal] * (len(train_test)-1)

In [ ]:
#train_test.loc[:,['signal']] = [signal] * (len(train_test))
# train_test.loc[:,['signal']] = [signal]
# df.insert(0, 'A', 'foo')
train_test.insert(0, 'signal', signal)

In [ ]:
train_test.tail(6)

In [ ]:
# start=len(train)
# end=len(train)+len(test)-1
start = len(train_test)-5
end = len(train_test)-1
start, end

In [ ]:
# Set for testing
train = train_test.iloc[:start]
test = train_test.iloc[start:]

In [ ]:
test.head()

In [ ]:
def train_test_plot(train, test):
    plt.figure(figsize=(16, 8))
    plt.plot(train, c='blue')
    plt.plot(test, c='orange');

In [ ]:
# This plot confirms that our train test split makes sense
train_test_plot(train['close'], test['close'])

In [ ]:
auto_arima(daily['close'].dropna(), seasonal=False).summary()

In [ ]:
train

In [ ]:
model = ARIMA(train['low'], order=(0,1,0))
results = model.fit()
results.summary()

In [ ]:
predictions = results.predict(start=start, end=end, dynamic=False, typ='levels').rename('ARIMA-0-1-0 Predictions')

In [ ]:
predictions.values

In [ ]:
type(predictions)

In [ ]:
def justified(row):
    
    if row['signal'] == -1 and row['low'] <= row['double_height']:
        return 1
    elif row['signal'] == 1 and row['high'] >= row['double_height']:
        return 1    
    else:
        return 0

In [ ]:
outcomes = pd.DataFrame()
outcomes['low'] = test['low']
outcomes['high'] = test['high']

outcomes['preds'] = predictions.values
outcomes['target_price'] = test['target_price']
# outcomes['direction'] = test['signal']
outcomes['signal_match'] = test.apply(justified, axis=1)

#daily_pre['double_height'] = daily_pre.apply(choose_exit_price, axis=1)
# outcomes.append(predictions, ignore_index=True)
outcomes

In [ ]:
# predictions['date']  = test.index
#predictions.reset_index(test.index)

In [ ]:
# predictions.reindex(test.index)

In [ ]:
type(predictions)

In [ ]:
test.head()['close'].isnull().sum()

In [ ]:
train.head()['close'].isnull().sum()

In [ ]:
outcomes['low'].plot(legend=True, figsize=(12,8))
outcomes['preds'].plot(legend=True);
outcomes['target_price'].plot(legend=True);

# predictions.plot(legend=True)

In [ ]:

error = mean_squared_error(test['close'], predictions)
print(f'ARIMA(0,1,0) MSE Error: {error:11.10}')


error = rmse(test['close'], predictions)
print(f'ARIMA(0,1,0) RMSE Error: {error:11.10}')

In [ ]:
results = {'algo':'','name':'','date':'', 'time_frame':'','success':'','RMSE':'', 'MSE':'', 'classification':'' }


In [ ]:
daily.columns

---

# SARIMAX


In [ ]:
# daily = daily.resample('B').agg({'open':'first','high':'max',
#                                         'low':'min', 'close':'last'})

In [ ]:
daily.index

In [ ]:
daily['close'].dropna(inplace=True)

In [ ]:
result = seasonal_decompose(daily['close'], model='add', period=400 )
result.plot();

In [ ]:
%%time
auto_arima(daily['close'], seasonal=True, maxiter=10000).summary()

In [ ]:
model = SARIMAX(train['close'], order=(0,1,0), seasonal_order=(1,0,1,12))

In [ ]:
len(train)

In [ ]:
train.columns

In [ ]:
# Starting MSE and (P, D, Q).
mse = 99 * (10 ** 16)
final_P = 0
final_D = 0
final_Q = 0

for P in range(3):
    for Q in range(3):
        for D in range(3):
            try:
                # Instantiate SARIMA model.
                sarima = SARIMAX(endog = train['close'],
                                 order = (0, 1, 0),              # (p, d, q)
                                 seasonal_order = (P, D, Q, 12)) # (P, D, Q, S)

                # Fit SARIMA model.
                model = sarima.fit()

                # Generate predictions based on training set.
                # Start at time period 0 and end at 1028.
                preds = model.predict(start=0, end=1028)

                # Evaluate predictions.
                print(f'The MSE for (1, 0, 0)x({P},{D},{Q},12) is: {mean_squared_error(train["close"], preds)}')
                
                # Save for final report.
                if mse > mean_squared_error(train['close'], preds):
                    mse = mean_squared_error(train['close'], preds)
                    final_P = P
                    final_D = D
                    final_Q = Q
                
            except:
                print(f"p: {P}, D: {D}, Q: {Q}")
                pass

print(f'Our model that minimizes MSE on the training data is the SARIMA(1, 0, 0)x({final_P},{final_D},{final_Q},420).')
print(f'This model has an MSE of {mse}.')